In [1]:
import os

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

import json

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = =  = = =  = = =

from bs4 import BeautifulSoup
from lxml import etree

In [2]:
directory_in = './file/2.part'
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
file = list_file[0]

file

'1.xlsx'

In [3]:
print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：353



,No,Group,Type,Manufacturer,Part Number,OEM,Picture,Url,Json_Src,Info,Json_Note 1,Json_Note 2,Choose,Json_Price,Type Code,Part Code,JOIN_MPNTCPC
0,1,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APK100,15177923RK,,https://www.rockauto.com/en/parts/standard+mot...,"{""0"": ""https://www.rockauto.com/info/154/APK10...",https://www.rockauto.com/en/moreinfo.php?pk=12...,{},"{""0"": ""Category: Accelerator Pedal Position Se...",,"{""0"": ""$32.79""}",5061,12769033,STANDARD MOTOR PRODUCTS;APK100;5061;12769033
1,2,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APK101,15264643RK,,https://www.rockauto.com/en/parts/standard+mot...,"{""0"": ""https://www.rockauto.com/info/154/APK10...",https://www.rockauto.com/en/moreinfo.php?pk=12...,{},"{""0"": ""Category: Accelerator Pedal Position Se...",,"{""0"": ""$34.79""}",5061,12769037,STANDARD MOTOR PRODUCTS;APK101;5061;12769037
2,3,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APK102,15751307RK,,https://www.rockauto.com/en/parts/standard+mot...,"{""0"": ""https://www.rockauto.com/info/154/APK10...",https://www.rockauto.com/en/moreinfo.php?pk=12...,{},"{""0"": ""Category: Accelerator Pedal Position Se...",,"{""0"": ""$66.99""}",5061,12769041,STANDARD MOTOR PRODUCTS;APK102;5061;12769041
3,4,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APK103,F6TZ9F836AARK,,https://www.rockauto.com/en/parts/standard+mot...,"{""0"": ""https://www.rockauto.com/info/154/APK10...",https://www.rockauto.com/en/moreinfo.php?pk=13...,{},"{""0"": ""Category: Accelerator Pedal Position Se...",,"{""0"": ""$44.79""}",5061,13953693,STANDARD MOTOR PRODUCTS;APK103;5061;13953693
4,5,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APS101,F6TZ9F836AA,,https://www.rockauto.com/en/parts/standard+mot...,"{""0"": ""https://www.rockauto.com/info/154/APS10...",https://www.rockauto.com/en/moreinfo.php?pk=15...,{},"{""0"": ""Category: Accelerator Pedal Position Se...",,"{""0"": ""$71.79""}",5061,1513520,STANDARD MOTOR PRODUCTS;APS101;5061;1513520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,349,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APS713,7811048120;7811048121,,https://www.rockauto.com/en/parts/standard+mot...,"{""0"": ""https://www.rockauto.com/info/154/APS71...",https://www.rockauto.com/en/moreinfo.php?pk=15...,{},"{""0"": ""Category: Accelerator Pedal Position Se...",,"{""0"": ""$191.79""}",5061,15195357,STANDARD MOTOR PRODUCTS;APS713;5061;15195357
349,350,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APS714,7811060230,,https://www.rockauto.com/en/parts/standard+mot...,"{""0"": ""https://www.rockauto.com/info/154/APS71...",https://www.rockauto.com/en/moreinfo.php?pk=15...,{},"{""0"": ""Category: Accelerator Pedal Position Se...",,"{""0"": ""$303.79""}",5061,15195361,STANDARD MOTOR PRODUCTS;APS714;5061;15195361
350,351,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APS716,7811035010;7811035011;7811035020;7811035040,,https://www.rockauto.com/en/parts/standard+mot...,"{""0"": ""https://www.rockauto.com/info/154/APS71...",https://www.rockauto.com/en/moreinfo.php?pk=15...,{},"{""0"": ""Category: Accelerator Pedal Position Se...",,"{""0"": ""$176.89""}",5061,15195365,STANDARD MOTOR PRODUCTS;APS716;5061;15195365
351,352,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APS717,7811060220,,https://www.rockauto.com/en/parts/standard+mot...,"{""0"": ""https://www.rockauto.com/info/154/APS71...",https://www.rockauto.com/en/moreinfo.php?pk=14...,{},"{""0"": ""Category: Accelerator Pedal Position Se...",,"{""0"": ""$227.79""}",5061,14871709,STANDARD MOTOR PRODUCTS;APS717;5061;14871709


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                              1
 Group                                                    Interior
 Type                            Accelerator Pedal Position Sensor
 Manufacturer                              STANDARD MOTOR PRODUCTS
 Part Number                                                APK100
 OEM                                                    15177923RK
 Picture                                                          
 Url             https://www.rockauto.com/en/parts/standard+mot...
 Json_Src        {"0": "https://www.rockauto.com/info/154/APK10...
 Info            https://www.rockauto.com/en/moreinfo.php?pk=12...
 Json_Note 1                                                    {}
 Json_Note 2     {"0": "Category: Accelerator Pedal Position Se...
 Choose                                                           
 Json_Price                                        {"0": "$32.79"}
 Type Code                                        

In [5]:
import time

data = {'func': 'getbuyersguide',
        'payload': json.dumps({'partData': {'listing_data_essential': {'parttype': crawler_series['Type Code'],
                                                                       'partkey': crawler_series['Part Code']},
                                            'listing_data_supplemental': {'partnumber': crawler_series['Part Number'],
                                                                          'catalogname': crawler_series['Manufacturer']}}}),
        'api_json_request': '1'}

crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        url_check = crawler_series['Url']

        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                             data=data,
                             headers=get_header(),
                             proxies=get_proxy(),
                             timeout=(5, 15))

        if resp.status_code == 200:
            dict_ = resp.json()

            if 'redirect_to_url' not in dict_ and dict_['buyersguidepieces']['title'].startswith('Buyer\'s Guide :') and 'collected_javascript' in dict_:
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

dict_

1


{'buyersguidepieces': {'body': '<div class="buyersguide-nested"><div style="padding: .5em;"><table class="nobmp" style="width: 100%;"><tr><td class="altrow-a-0">CADILLAC</td><td class="altrow-a-0">ESCALADE</td><td class="altrow-a-0">2003</td></tr><tr><td class="altrow-a-1">CHEVROLET</td><td class="altrow-a-1">AVALANCHE 1500</td><td class="altrow-a-1">2003</td></tr><tr><td class="altrow-a-0">CHEVROLET</td><td class="altrow-a-0">AVALANCHE 2500</td><td class="altrow-a-0">2003</td></tr><tr><td class="altrow-a-1">CHEVROLET</td><td class="altrow-a-1">SILVERADO 1500</td><td class="altrow-a-1">2003</td></tr><tr><td class="altrow-a-0">CHEVROLET</td><td class="altrow-a-0">SILVERADO 2500</td><td class="altrow-a-0">2003-2005</td></tr><tr><td class="altrow-a-1">CHEVROLET</td><td class="altrow-a-1">SILVERADO 3500</td><td class="altrow-a-1">2003-2005</td></tr><tr><td class="altrow-a-0">CHEVROLET</td><td class="altrow-a-0">SUBURBAN 1500</td><td class="altrow-a-0">2003</td></tr><tr><td class="altrow-a-

In [6]:
soup = BeautifulSoup(dict_['buyersguidepieces']['body'], 'lxml')
html = etree.HTML(str(soup))

with open('./vehicle.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <div class="buyersguide-nested">
   <div style="padding: .5em;">
    <table class="nobmp" style="width: 100%;">
     <tr>
      <td class="altrow-a-0">
       CADILLAC
      </td>
      <td class="altrow-a-0">
       ESCALADE
      </td>
      <td class="altrow-a-0">
       2003
      </td>
     </tr>
     <tr>
      <td class="altrow-a-1">
       CHEVROLET
      </td>
      <td class="altrow-a-1">
       AVALANCHE 1500
      </td>
      <td class="altrow-a-1">
       2003
      </td>
     </tr>
     <tr>
      <td class="altrow-a-0">
       CHEVROLET
      </td>
      <td class="altrow-a-0">
       AVALANCHE 2500
      </td>
      <td class="altrow-a-0">
       2003
      </td>
     </tr>
     <tr>
      <td class="altrow-a-1">
       CHEVROLET
      </td>
      <td class="altrow-a-1">
       SILVERADO 1500
      </td>
      <td class="altrow-a-1">
       2003
      </td>
     </tr>
     <tr>
      <td class="altrow-a-0">
       CHEVROLET
      </td>
      <td class="

In [7]:
list_tr = html.xpath('//tr')

list_tr

[<Element tr at 0x11d096840>,
 <Element tr at 0x11deefe00>,
 <Element tr at 0x11dec0500>,
 <Element tr at 0x11dec3340>,
 <Element tr at 0x11dec1b80>,
 <Element tr at 0x11dec3940>,
 <Element tr at 0x11dec3140>,
 <Element tr at 0x11dec3a00>,
 <Element tr at 0x11df867c0>,
 <Element tr at 0x11dec2b00>,
 <Element tr at 0x11df87240>,
 <Element tr at 0x11df840c0>,
 <Element tr at 0x11df86c00>,
 <Element tr at 0x11df87f40>,
 <Element tr at 0x11df858c0>,
 <Element tr at 0x11df85440>]

In [8]:
not list_tr and not (html.xpath('//p/text()') and html.xpath('//p/text()')[0].strip() == 'No applications found.')

False

In [9]:
list_dict_correct_temp = []
for i, tr in enumerate(list_tr):
    list_text = [text.strip() for text in tr.xpath('./td/text()')]
    list_year = list_text[-1].split('-')
    year_begin, year_end = int(list_year[0]), int(list_year[-1])

    if len(list_text) in [2, 3]:
        list_dict_correct_temp.extend([{'JOIN_MPNTCPC': crawler_series['JOIN_MPNTCPC'],
                                        'Vehicle No': i + 1,
                                        'Make': list_text[0],
                                        'Model': list_text[1] if len(list_text) == 3 else '',
                                        'Year': year} for year in range(year_begin, year_end + 1)])
    else:
        print('fuck')

list_dict_correct_temp

[{'JOIN_MPNTCPC': 'STANDARD MOTOR PRODUCTS;APK100;5061;12769033',
  'Vehicle No': 1,
  'Make': 'CADILLAC',
  'Model': 'ESCALADE',
  'Year': 2003},
 {'JOIN_MPNTCPC': 'STANDARD MOTOR PRODUCTS;APK100;5061;12769033',
  'Vehicle No': 2,
  'Make': 'CHEVROLET',
  'Model': 'AVALANCHE 1500',
  'Year': 2003},
 {'JOIN_MPNTCPC': 'STANDARD MOTOR PRODUCTS;APK100;5061;12769033',
  'Vehicle No': 3,
  'Make': 'CHEVROLET',
  'Model': 'AVALANCHE 2500',
  'Year': 2003},
 {'JOIN_MPNTCPC': 'STANDARD MOTOR PRODUCTS;APK100;5061;12769033',
  'Vehicle No': 4,
  'Make': 'CHEVROLET',
  'Model': 'SILVERADO 1500',
  'Year': 2003},
 {'JOIN_MPNTCPC': 'STANDARD MOTOR PRODUCTS;APK100;5061;12769033',
  'Vehicle No': 5,
  'Make': 'CHEVROLET',
  'Model': 'SILVERADO 2500',
  'Year': 2003},
 {'JOIN_MPNTCPC': 'STANDARD MOTOR PRODUCTS;APK100;5061;12769033',
  'Vehicle No': 5,
  'Make': 'CHEVROLET',
  'Model': 'SILVERADO 2500',
  'Year': 2004},
 {'JOIN_MPNTCPC': 'STANDARD MOTOR PRODUCTS;APK100;5061;12769033',
  'Vehicle No': 5

In [10]:
list_dict_correct.extend(list_dict_correct_temp)

list_dict_correct

[{'JOIN_MPNTCPC': 'STANDARD MOTOR PRODUCTS;APK100;5061;12769033',
  'Vehicle No': 1,
  'Make': 'CADILLAC',
  'Model': 'ESCALADE',
  'Year': 2003},
 {'JOIN_MPNTCPC': 'STANDARD MOTOR PRODUCTS;APK100;5061;12769033',
  'Vehicle No': 2,
  'Make': 'CHEVROLET',
  'Model': 'AVALANCHE 1500',
  'Year': 2003},
 {'JOIN_MPNTCPC': 'STANDARD MOTOR PRODUCTS;APK100;5061;12769033',
  'Vehicle No': 3,
  'Make': 'CHEVROLET',
  'Model': 'AVALANCHE 2500',
  'Year': 2003},
 {'JOIN_MPNTCPC': 'STANDARD MOTOR PRODUCTS;APK100;5061;12769033',
  'Vehicle No': 4,
  'Make': 'CHEVROLET',
  'Model': 'SILVERADO 1500',
  'Year': 2003},
 {'JOIN_MPNTCPC': 'STANDARD MOTOR PRODUCTS;APK100;5061;12769033',
  'Vehicle No': 5,
  'Make': 'CHEVROLET',
  'Model': 'SILVERADO 2500',
  'Year': 2003},
 {'JOIN_MPNTCPC': 'STANDARD MOTOR PRODUCTS;APK100;5061;12769033',
  'Vehicle No': 5,
  'Make': 'CHEVROLET',
  'Model': 'SILVERADO 2500',
  'Year': 2004},
 {'JOIN_MPNTCPC': 'STANDARD MOTOR PRODUCTS;APK100;5061;12769033',
  'Vehicle No': 5

In [11]:
crawler_status = True

crawler_status

True

In [12]:
crawler_series['Check_Url'] = url_check

list_series_error.append(crawler_series)

list_series_error

[No                                                              1
 Group                                                    Interior
 Type                            Accelerator Pedal Position Sensor
 Manufacturer                              STANDARD MOTOR PRODUCTS
 Part Number                                                APK100
 OEM                                                    15177923RK
 Picture                                                          
 Url             https://www.rockauto.com/en/parts/standard+mot...
 Json_Src        {"0": "https://www.rockauto.com/info/154/APK10...
 Info            https://www.rockauto.com/en/moreinfo.php?pk=12...
 Json_Note 1                                                    {}
 Json_Note 2     {"0": "Category: Accelerator Pedal Position Se...
 Choose                                                           
 Json_Price                                        {"0": "$32.79"}
 Type Code                                                    

In [13]:
print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['JOIN_MPNTCPC', 'Vehicle No', 'Year'],
                                                             ascending=[True, True, False],
                                                             ignore_index=True)
    df_correct.to_excel('./test_compatibility.xlsx', index=False)

df_correct if list_dict_correct else ''

Data Outputting...


,JOIN_MPNTCPC,Vehicle No,Make,Model,Year
0,STANDARD MOTOR PRODUCTS;APK100;5061;12769033,1,CADILLAC,ESCALADE,2003
1,STANDARD MOTOR PRODUCTS;APK100;5061;12769033,2,CHEVROLET,AVALANCHE 1500,2003
2,STANDARD MOTOR PRODUCTS;APK100;5061;12769033,3,CHEVROLET,AVALANCHE 2500,2003
3,STANDARD MOTOR PRODUCTS;APK100;5061;12769033,4,CHEVROLET,SILVERADO 1500,2003
4,STANDARD MOTOR PRODUCTS;APK100;5061;12769033,5,CHEVROLET,SILVERADO 2500,2005
5,STANDARD MOTOR PRODUCTS;APK100;5061;12769033,5,CHEVROLET,SILVERADO 2500,2004
6,STANDARD MOTOR PRODUCTS;APK100;5061;12769033,5,CHEVROLET,SILVERADO 2500,2003
7,STANDARD MOTOR PRODUCTS;APK100;5061;12769033,6,CHEVROLET,SILVERADO 3500,2005
8,STANDARD MOTOR PRODUCTS;APK100;5061;12769033,6,CHEVROLET,SILVERADO 3500,2004
9,STANDARD MOTOR PRODUCTS;APK100;5061;12769033,6,CHEVROLET,SILVERADO 3500,2003


In [14]:
print('Output Finished !')
print()

Output Finished !



In [15]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_compatibility_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Group,Type,Manufacturer,Part Number,OEM,Picture,Url,Json_Src,Info,Json_Note 1,Json_Note 2,Choose,Json_Price,Type Code,Part Code,JOIN_MPNTCPC,Check_Url
0,1,Interior,Accelerator Pedal Position Sensor,STANDARD MOTOR PRODUCTS,APK100,15177923RK,,https://www.rockauto.com/en/parts/standard+mot...,"{""0"": ""https://www.rockauto.com/info/154/APK10...",https://www.rockauto.com/en/moreinfo.php?pk=12...,{},"{""0"": ""Category: Accelerator Pedal Position Se...",,"{""0"": ""$32.79""}",5061,12769033,STANDARD MOTOR PRODUCTS;APK100;5061;12769033,https://www.rockauto.com/en/parts/standard+mot...


In [16]:
print('Done ~')

Done ~
